In [1]:
import pandas as pd
import numpy as np
from ML_pr import *

In [2]:
crime_data_2021_present = pd.read_csv('csv_files/Crimes_2021_to_Present.csv')

ML - 1. Predict if a person would get arrested considering primary type of crime, location description and neighborhood (region name)


# Best Feature

In [3]:
X = crime_data_2021_present[['Primary Type', 'Location Description', 'RegionName']]
y = crime_data_2021_present['Arrest'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(features, accuracy) = feature_selection_and_evaluation(X_train, y_train)


In [8]:
print(features, accuracy)

['Primary Type', 'Location Description', 'RegionName'] 0.9112935253071257


# Training/Testing the model
<h5>using the best feature, train the LogisticRegression Model using the same training set that we did to get the best features.
Test the model on the testing set <h5>

In [24]:
# Training and Testing Data Split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

encoder = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories by ignoring them

# Training: Encode features and fit the encoder
X_train_encoded = encoder.fit_transform(X_train[features])
X_test_encoded = encoder.transform(X_train['Primary Type'])

# Testing: Use the same encoder to transform features
X_test_encoded = encoder.transform(X_test[features])
X_test_encoded = encoder.transform(X_test['Primary Type'])

# Create and train the logistic regression model
model = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', C=1.0)  
model.fit(X_train_encoded, y_train)

# Predict on the testing dataset
y_pred_test = model.predict(X_test_encoded)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred_test)

print("Accuracy:", accuracy)





Accuracy: 0.9119251158788754


In [ ]:
# Highly Recommended for Final report ()
# 2 data (Crime and Housing)

# You want to get a mean house price of all the region in Housing data (181 columns and 1 row) 
# Rotate teh data (2 columns)
# Zone for different housing prices (Ex 10-50,000 Zone 1, 50-100,000 Zone 2, so on)
# Join them (Housing Data and Crime) on Region Name  => New Data (Save CSV)
# Create a ML Model Classificatio (SVM, NN)
# Using the Zone, Region Name, Primary Type, Location Description => Will the person be arrested or not (0,1)


In [ ]:
# Do T-Test to test our hypothesis